In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.4)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.74 sec - Loss 0.090028 - ACC 71.04% - ACC Mean 71.04% - AUC 77.28% - AUC Mean 77.28% - Deter 000
Ite 00050 - 0.11 sec - Loss 0.085444 - ACC 72.57% - ACC Mean 70.25% - AUC 78.85% - AUC Mean 77.81% - Deter 034
Ite 00100 - 0.54 sec - Loss 0.074959 - ACC 71.01% - ACC Mean 71.13% - AUC 77.72% - AUC Mean 78.40% - Deter 015
Ite 00150 - 0.11 sec - Loss 0.064008 - ACC 69.42% - ACC Mean 71.01% - AUC 76.64% - AUC Mean 78.15% - Deter 020
Ite 00200 - 0.44 sec - Loss 0.060939 - ACC 70.40% - ACC Mean 70.54% - AUC 77.10% - AUC Mean 77.49% - Deter 070
Ite 00250 - 0.11 sec - Loss 0.060184 - ACC 72.27% - ACC Mean 70.42% - AUC 78.26% - AUC Mean 77.41% - Deter 120
Ite 00300 - 0.33 sec - Loss 0.059803 - ACC 72.22% - ACC Mean 70.77% - AUC 79.02% - AUC Mean 77.64% - Deter 170
Ite 00350 - 0.13 sec - Loss 0.059436 - ACC 70.71% - ACC Mean 71.31% - AUC 77.81% - AUC Mean 78.17% - Deter 220
Ite 00400 - 0.12 sec - Loss 0.059059 - ACC 69.33% - ACC Mean 71.30% - AUC 76.72% - AUC Mean 78.19% - Deter 270
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 68.05% +- 0.0% - auc: 76.38% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 70.37% +- 3.65% - auc: 72.61% +- 4.73%
SFIL RUN 2/10 - acc: 66.98% +- 5.52% - auc: 70.55% +- 3.61%
SFIL RUN 3/10 - acc: 65.53% +- 5.3% - auc: 72.11% +- 8.87%
SFIL RUN 4/10 - acc: 66.5% +- 5.38% - auc: 75.31% +- 7.21%
SFIL RUN 5/10 - acc: 67.66% +- 6.74% - auc: 73.06% +- 3.49%
SFIL RUN 6/10 - acc: 67.41% +- 4.13% - auc: 69.66% +- 5.23%
SFIL RUN 7/10 - acc: 72.0% +- 3.62% - auc: 75.68% +- 5.27%
SFIL RUN 8/10 - acc: 69.59% +- 6.29% - auc: 73.08% +- 6.7%
SFIL RUN 9/10 - acc: 68.95% +- 3.28% - auc: 72.95% +- 3.94%
SFIL RUN 10/10 - acc: 65.09% +- 8.0% - auc: 71.36% +- 9.88%
SFIL GLOBAL - acc: 68.01% +- 2.08% - auc: 72.64% +- 1.79%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 65.88% +- 4.61% - auc: 70.25% +- 4.5%
PFIL RUN 2/10 - acc: 65.88% +- 3.91% - auc: 70.02% +- 4.47%
PFIL RUN 3/10 - acc: 64.72% +- 3.78% - auc: 74.19% +- 2.92%
PFIL RUN 4/10 - acc: 67.26% +- 7.24% - auc: 73.05% +- 6.29%
PFIL RUN 5/10 - acc: 71.08% +- 3.05% - auc: 76.37% +- 5.16%
PFIL RUN 6/10 - acc: 71.01% +- 4.54% - auc: 74.05% +- 7.06%
PFIL RUN 7/10 - acc: 69.88% +- 6.39% - auc: 75.39% +- 7.73%
PFIL RUN 8/10 - acc: 70.06% +- 5.42% - auc: 76.56% +- 7.58%
PFIL RUN 9/10 - acc: 71.53% +- 6.7% - auc: 77.18% +- 4.4%
PFIL RUN 10/10 - acc: 69.74% +- 5.73% - auc: 74.64% +- 6.81%
PFIL GLOBAL - acc: 68.7% +- 2.39% - auc: 74.17% +- 2.35%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 65.23% +- 5.88% - auc: 72.03% +- 7.28%
SPOL RUN 2/10 - acc: 64.31% +- 2.94% - auc: 69.29% +- 4.71%
SPOL RUN 3/10 - acc: 67.2% +- 4.85% - auc: 72.63% +- 3.99%
SPOL RUN 4/10 - acc: 69.65% +- 5.52% - auc: 72.48% +- 6.78%
SPOL RUN 5/10 - acc: 69.41% +- 8.05% - auc: 72.37% +- 8.32%
SPOL RUN 6/10 - acc: 68.46% +- 4.96% - auc: 73.91% +- 6.31%
SPOL RUN 7/10 - acc: 63.85% +- 10.68% - auc: 70.44% +- 8.54%
SPOL RUN 8/10 - acc: 72.71% +- 5.0% - auc: 76.4% +- 5.94%
SPOL RUN 9/10 - acc: 64.91% +- 5.07% - auc: 71.55% +- 4.62%
SPOL RUN 10/10 - acc: 64.23% +- 4.86% - auc: 69.81% +- 7.3%
SPOL GLOBAL - acc: 67.0% +- 2.83% - auc: 72.09% +- 1.96%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 70.99% +- 3.71% - auc: 76.57% +- 5.63%
PPOL RUN 2/10 - acc: 68.8% +- 3.82% - auc: 75.09% +- 3.48%
PPOL RUN 3/10 - acc: 70.52% +- 5.4% - auc: 76.66% +- 5.3%
PPOL RUN 4/10 - acc: 69.91% +- 4.96% - auc: 76.21% +- 5.06%
PPOL RUN 5/10 - acc: 67.78% +- 3.46% - auc: 75.62% +- 3.09%
PPOL RUN 6/10 - acc: 68.4% +- 3.27% - auc: 74.09% +- 3.98%
PPOL RUN 7/10 - acc: 67.98% +- 9.11% - auc: 72.15% +- 7.97%
PPOL RUN 8/10 - acc: 71.2% +- 4.42% - auc: 76.75% +- 6.98%
PPOL RUN 9/10 - acc: 68.74% +- 5.75% - auc: 74.99% +- 6.18%
PPOL RUN 10/10 - acc: 70.4% +- 4.69% - auc: 77.49% +- 1.77%
PPOL GLOBAL - acc: 69.47% +- 1.21% - auc: 75.56% +- 1.49%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
